##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Universal Sentence Encoder


<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/hub/blob/master/examples/colab/semantic_similarity_with_tf_hub_universal_encoder.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>


This notebook illustrates how to access the Universal Sentence Encoder and use it for sentence similarity and sentence classification tasks.

The Universal Sentence Encoder makes getting sentence level embeddings as easy as it has historically been to lookup the embeddings for individual words. The sentence embeddings can then be trivially used to compute sentence level meaning similarity as well as to enable better performance on downstream classification tasks using less supervised training data.


# Getting Started

This section sets up the environment for access to the Universal Sentence Encoder on TF Hub and provides examples of applying the encoder to words, sentences, and paragraphs.

In [ ]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

More detailed information about installing Tensorflow can be found at [https://www.tensorflow.org/install/](https://www.tensorflow.org/install/).

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import json as json

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
similarity_data = np.array([['ticker', 'item_type', 'prev_year', 'curr_year', 'similarity_score']])

In [3]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [4]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

INFO:tensorflow:Using /Users/kashishgarg/tensorflow_module_cache to cache modules.


In [5]:
#read file 1 section 1A text
file = open("./data/downloaded_files/AME/2016/AME_0001037868_20161231_Item1A.txt", "r")
text1 = file.read()
text1Lines = text1.split(".")

In [6]:
#read file 1 section 1A text
file = open("./data/downloaded_files/AME/2017/AME_0001037868_20171231_Item1A.txt", "r")
text2 = file.read()
text2Lines = text2.split(".")

In [7]:
path = '/Users/kashishgarg/git/similarity_analysis/data/downloaded_files'
company_tickers = []
# for filename in os.listdir(path):
#     company_tickers.append(filename)
    
company_tickers = [filename for filename in os.listdir(path) if filename != '.DS_Store']

company_sections_dict = {}
for company in company_tickers:
    item1_dict = {}
    item1a_dict = {}
    item7_dict = {}
    item7a_dict = {}
    company_file_path = path + '/' + company
    years = [filename for filename in os.listdir(company_file_path) if filename != '.DS_Store']
    for report_year in years:
        section_file_path = company_file_path + '/' + report_year
        for filename in os.listdir(section_file_path):
            if('Item1A' in filename):
                item1a_dict[report_year] = open(section_file_path + '/' + filename, "r").read()
            elif ('Item1' in filename):
                item1_dict[report_year] = open(section_file_path + '/' + filename, "r").read()
            elif ('Item7A' in filename):
                item7a_dict[report_year] = open(section_file_path + '/' + filename, "r").read()
            elif ('Item7' in filename):
                item7_dict[report_year] = open(section_file_path + '/' + filename, "r").read()
    company_sections_dict[company] = {'item1': item1_dict, 'item1a': item1a_dict, 'item7': item7_dict, 'item7a': item7a_dict}
# df = pd.DataFrame.from_dict(company_sections_dict, orient='columns')
# df = pd.DataFrame.from_dict({(i,j): company_sections_dict[i][j] 
#                            for i in company_sections_dict.keys() 
#                            for j in company_sections_dict[i].keys()},
#                        orient='columns')
df = pd.DataFrame(company_sections_dict).stack().apply(pd.Series).unstack()

In [12]:
sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_input2 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1), axis=1)
sts_encode2 = tf.nn.l2_normalize(embed(sts_input2), axis=1)
cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
sim_scores = 1.0 - tf.acos(clip_cosine_similarities)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [13]:
similarity_score_path = '/Users/kashishgarg/git/similarity_analysis/data/similarity_scores'

In [18]:
print(company_sections_dict.get('AAL').get('item1').get(2007))

None


In [23]:
report_years = ['2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018']
def run_similarity(session, text_across_years):
  """Returns the similarity scores"""
      
  emba, embb, scores = session.run(
      [sts_encode1, sts_encode2, sim_scores],
      feed_dict={
          sts_input1: [text_across_years.get(report_years[i], '') for i in range( len(report_years) -1)],
          sts_input2: [text_across_years.get(report_years[i], '') for i in range(1, len(report_years))]
      })
  scores_list = scores.tolist()
  item_similarity = []
  for i in range(len(report_years) -1):
    if scores_list[i] < 0:
        item_similarity.append({report_years[i] + '-' + report_years[i+1]: 0})
    elif scores_list[i] > 1:
        item_similarity.append({report_years[i] + '-' + report_years[i+1]: 1})
    else:
        item_similarity.append({report_years[i] + '-' + report_years[i+1]: scores_list[i]})
  return item_similarity


for company in company_sections_dict.keys():
    print("calculating similarity for: " + company)
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
        if company_sections_dict.get(company, {}).get('item1') is not None:
            item1_similarity = run_similarity(session, company_sections_dict.get(company).get('item1'))
        if company_sections_dict.get(company, {}).get('item1a') is not None:
            item1a_similarity = run_similarity(session, company_sections_dict.get(company).get('item1a'))
        if company_sections_dict.get(company, {}).get('item7') is not None:
            item7_similarity = run_similarity(session, company_sections_dict.get(company).get('item7'))
        if company_sections_dict.get(company, {}).get('item7a') is not None:
            item7a_similarity = run_similarity(session, company_sections_dict.get(company).get('item7a'))
        similarity_dict = {'item1': item1_similarity, 'item1a': item1a_similarity, 
                           'item7': item7_similarity, 'item7a': item7a_similarity}
        json_string = json.dumps(similarity_dict)
        open(similarity_score_path + '/' + company + '_score.json', 'w').write(json_string)
    print('completed writing similarity scores for: ' + company)

calculating similarity for: AAL
completed writing similarity scores for: AAL
calculating similarity for: AME
completed writing similarity scores for: AME
calculating similarity for: AON
completed writing similarity scores for: AON
calculating similarity for: AXP
completed writing similarity scores for: AXP
calculating similarity for: BAX
completed writing similarity scores for: BAX
calculating similarity for: BBT
completed writing similarity scores for: BBT
calculating similarity for: BK
completed writing similarity scores for: BK
calculating similarity for: BRKB
completed writing similarity scores for: BRKB
calculating similarity for: C
completed writing similarity scores for: C
calculating similarity for: CAT
completed writing similarity scores for: CAT
calculating similarity for: CBS
completed writing similarity scores for: CBS
calculating similarity for: CCI
completed writing similarity scores for: CCI
calculating similarity for: CL
completed writing similarity scores for: CL
calcu

In [94]:
print(scores)

[ 0.14230835 -0.11991525 -0.41105306]


In [6]:
# Import the Universal Sentence Encoder's TF Hub module
# embed = hub.Module(module_url)

messages = [text1, text2]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
#     print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Embedding size: 512
Embedding: [-0.03950010985136032, 0.01821286231279373, 0.0436711348593235, ...]

Embedding size: 512
Embedding: [-0.034965552389621735, 0.043505389243364334, 0.04279985651373863, ...]



In [7]:
embeddings_2007 = np.array(message_embeddings).tolist()[0]
embeddings_2008 = np.array(message_embeddings).tolist()[1]
np.matmul(embeddings_2007, embeddings_2008)

0.97456092929988

In [ ]:
# embeddings_by_line_2007 = np.array(message_embeddings).tolist()[2]
# embeddings_by_line_2008 = np.array(message_embeddings).tolist()[3]
# np.matmul(embeddings_by_line_2007, embeddings_by_line_2007)

In [ ]:
nnlm_module_url = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/1"
nnlm_embed = hub.Module(nnlm_module_url)
nnlm_embeddings = nnlm_embed([text1, text2])

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  nnlm_message_embeddings = session.run(nnlm_embeddings)
  print(type(nnlm_message_embeddings))

  for i, nnlm_message_embedding in enumerate(np.array(nnlm_message_embeddings).tolist()):
#     print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(nnlm_message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in nnlm_message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))
    
mean = np.mean(nnlm_message_embeddings)
year1_embedding = np.array(nnlm_message_embeddings/128).tolist()[0]
year2_embedding = np.array(nnlm_message_embeddings/128).tolist()[1]
print(np.matmul(year1_embedding, year2_embedding))

In [ ]:
# elmo_module_url = "https://tfhub.dev/google/elmo/2"
# elmo = hub.Module(elmo_module_url, trainable=False)
# elmo_embeddings = elmo(
#     [text1, text2])
# #     signature="default",
# #     as_dict=True)["elmo"]
# with tf.Session() as session:
#   session.run([tf.global_variables_initializer(), tf.tables_initializer()])
#   elmo_message_embeddings = session.run(elmo_embeddings)
    
# for i, elmo_message_embedding in enumerate(np.array(elmo_message_embeddings).tolist()):
# #     print("Message: {}".format(messages[i]))
#     print("Embedding size: {}".format(len(elmo_message_embedding)))
#     message_embedding_snippet = ", ".join(
#         (str(x) for x in nnlm_message_embedding[:3]))
#     print("Embedding: [{}, ...]\n".format(message_embedding_snippet))
    

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


# Semantic Textual Similarity Task Example

The embeddings produced by the Universal Sentence Encoder are approximately normalized. The semantic similarity of two sentences can be trivially computed as the inner product of the encodings.

In [ ]:
def plot_similarity(labels, features, rotation):
  corr = np.inner(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      cmap="YlOrRd")
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")


def run_and_plot(session_, input_tensor_, messages_, encoding_tensor):
  message_embeddings_ = session_.run(
      encoding_tensor, feed_dict={input_tensor_: messages_})
  plot_similarity(messages_, message_embeddings_, 90)

## Similarity Visualized
Here we show the similarity in a heat map. The final graph is a 9x9 matrix where each entry `[i, j]` is colored based on the inner product of the encodings for sentence `i` and `j`.

In [ ]:
messages = [
    # Smartphones
    "I like my phone",
    "My phone is not good.",
    "Your cellphone looks great.",

    # Weather
    "Will it snow tomorrow?",
    "Recently a lot of hurricanes have hit the US",
    "Global warming is real",

    # Food and health
    "An apple a day, keeps the doctors away",
    "Eating strawberries is healthy",
    "Is paleo better than keto?",

    # Asking about age
    "How old are you?",
    "what is your age?",
]

similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_message_encodings = embed(similarity_input_placeholder)
with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  run_and_plot(session, similarity_input_placeholder, messages,
               similarity_message_encodings)

## Evaluation: STS (Semantic Textual Similarity) Benchmark

The [**STS Benchmark**](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) provides an intristic evaluation of the degree to which similarity scores computed using sentence embeddings align with human judgements. The benchmark requires systems to return similarity scores for a diverse selection of sentence pairs. [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) is then used to evaluate the quality of the machine similarity scores against human judgements.

### Download data

In [10]:
import pandas
import scipy
import math


def load_sts_dataset(filename):
  # Loads a subset of the STS dataset into a DataFrame. In particular both
  # sentences and their human rated similarity score.
  sent_pairs = []
  with tf.gfile.GFile(filename, "r") as f:
    for line in f:
      ts = line.strip().split("\t")
      # (sent_1, sent_2, similarity_score)
      sent_pairs.append((ts[5], ts[6], float(ts[4])))
  return pandas.DataFrame(sent_pairs, columns=["sent_1", "sent_2", "sim"])


def download_and_load_sts_data():
  sts_dataset = tf.keras.utils.get_file(
      fname="Stsbenchmark.tar.gz",
      origin="http://ixa2.si.ehu.es/stswiki/images/4/48/Stsbenchmark.tar.gz",
      extract=True)

  sts_dev = load_sts_dataset(
      os.path.join(os.path.dirname(sts_dataset), "stsbenchmark", "sts-dev.csv"))
  sts_test = load_sts_dataset(
      os.path.join(
          os.path.dirname(sts_dataset), "stsbenchmark", "sts-test.csv"))

  return sts_dev, sts_test


sts_dev, sts_test = download_and_load_sts_data()

In [13]:
sts_data = sts_dev
print(type(sts_data['sent_1'].tolist()))

<class 'list'>


### Build Evaluation Graph

In [11]:
sts_input1 = tf.placeholder(tf.string, shape=(None))
sts_input2 = tf.placeholder(tf.string, shape=(None))

# For evaluation we use exactly normalized rather than
# approximately normalized.
sts_encode1 = tf.nn.l2_normalize(embed(sts_input1), axis=1)
sts_encode2 = tf.nn.l2_normalize(embed(sts_input2), axis=1)
cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
sim_scores = 1.0 - tf.acos(clip_cosine_similarities)

### Evaluate Sentence Embeddings

In [9]:
sts_data = sts_dev #@param ["sts_dev", "sts_test"] {type:"raw"}

NameError: name 'sts_dev' is not defined

In [14]:
text_a = sts_data['sent_1'].tolist()
text_b = sts_data['sent_2'].tolist()
dev_scores = sts_data['sim'].tolist()

def run_sts_benchmark(session):
  """Returns the similarity scores"""
  emba, embb, scores = session.run(
      [sts_encode1, sts_encode2, sim_scores],
      feed_dict={
          sts_input1: text_a,
          sts_input2: text_b
      })
  return scores


with tf.Session() as session:
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  scores = run_sts_benchmark(session)

print(scores)
pearson_correlation = scipy.stats.pearsonr(scores, dev_scores)
print('Pearson correlation coefficient = {0}\np-value = {1}'.format(
    pearson_correlation[0], pearson_correlation[1]))

[0.8127506  0.7951114  0.6856092  ... 0.02519631 0.08812553 0.29005396]
Pearson correlation coefficient = 0.7629205501666685
p-value = 4.521539751346317e-286
